# Networking for Big Data - Project
- Jonas Barth 2050678
- Susanna Bravi 1916681
- Eric Rubia Aguilera 2049558

# Setup

In [ ]:
import pyshark
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import copy
import sys
import shutil
import os
from functools import reduce
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm


Make all displayed dataframes interactive.

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

Autoreload external modules to avoid restarting the kernel every time changes are made.

In [ ]:
%load_ext autoreload
%autoreload 2

# Part A
#### 1. Extract general info from your trace using capinfos

In [ ]:
!capinfos -A data/packets.pcap

#### 2. Time Evaluation between Sequential and Parallel reading

We evaluate the time taken to process `.pcap` file sequentially and parallely as a function of the number of packets in the `.pcap` file. This allows us to show how the processing time scales with the number of packets. We initially used the `pyshark` library, however since it proved to be too slow to allow for a fast turnaround, we decided to use the `scapy` library instead which offers a similar functionality to `pyshark` but with better speed. Each processing algorithm is timed once, using Python's `timeit` package.
   
The [read.py](reading/) script lets you time either a sequential or parallel algorithm for a single `.pcap` file. The results are stored in a `.feather` file.,

We run the processing for the following number of packets:,

* 10
* 100
* 1000
* 10000
* 100000
* 1000000

##### Sequential Processing,
The sequential processing, opens the `.pcap` file, reads the packets one by one into domain objects, creates a dataframe, and closes the file.,

##### Parallel Processing,
The parallel processing algorithm first divides the given `.pcap` file into $n$ smaller `.pcap` files of $p$ packets each. Then, a maximum of $m$ processes are started in parallel, each of which processes a single `.pcap` file sequentially. The number of parallel processes is capped at the parameter $m$ as not to overwhelm the available computing resources.,

#### Results,
The sequential and parallel processing algorithms were run once for each `.pcap` file. The line plot below shows the processing time in seconds for the number of packets in the `.pcap` files. For files with a number of packets $\le 10000$, the parallel and sequential reading takes more or less the same amount of time. For the largest file of $1000000$ packets, the parallel processing is much faster than the sequential processing.,

With parallel processing, there is some overhead in splitting the original file and starting the processes, hence we can expect that the pay-off of parallel processing to be insignificant for a small number of packets.

First, we load the saved timing data and split the times into a sequential and parallel group.

In [ ]:
timing_df = pd.read_feather("data/reading_times.feather")
parallel_times = timing_df[timing_df.read_type == 'parallel']
sequential_times = timing_df[timing_df.read_type == 'sequential']

Then we plot using the function from our own `plot` package.

In [ ]:
import plot
fig, _ = plot.plot_read_time(sequential_times, parallel_times)
fig.show()

## 3. Extract the IP which generates the highest amount as sender traffic, evaluate the bit rate (0.1 sec) for the 6 IP addresses mostly used as endpoint


We have close to a million packets in our data.

In [ ]:
data_frame = pd.read_feather("data/packets_df.feather")
len(data_frame)

#### Highest Sender Traffic

Count and sort by the number of packets sent from each source address, to find the

In [ ]:
ip_grouped = data_frame.groupby(["IP_SRC"], as_index=False)[['length']].agg('sum')
ip_grouped.sort_values(by=['length'], ascending=False, inplace=True)
top_10_ip_src = ip_grouped.head(10)
max_ip = ip_grouped.iloc[0].IP_SRC

Plotting the top 10 senders.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.bar(x=top_10_ip_src.IP_SRC, height=top_10_ip_src.length, tick_label=top_10_ip_src.IP_SRC)
ax.set_yscale("log")
ax.set_title("Top 10 Senders: IP Addresses")
ax.set_yticks([10**y for y in range(6, 9)], [f"$10^{y}$" for y in range(6, 9)])
fig.show()

#### Evaluate Bit Rate

For finding the top 6 destination addresses, we group on the `IP_DST` column and sum the number of found rows. Then, we sort and pick the top 6.

In [ ]:
max_ip_data = data_frame[data_frame.IP_SRC == max_ip]
top_6_ip_dest = max_ip_data.groupby(["IP_DST"], as_index=False)[['length']].agg('sum').sort_values(by=['length'], ascending=False).head(6)

Plot the top 6 destination addresses.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.bar(x=top_6_ip_dest.IP_DST, height=top_6_ip_dest.length, tick_label=top_6_ip_dest.IP_DST)
ax.set_title("Top 6 Destinations: IP Addresses")
fig.show()

In [ ]:
fig = px.histogram(data_frame["length"], x='length',
                   nbins=15, 
                   title='Histogram of Packet Length',
                   labels={'length':'Packet Length (Byte)'},
                   opacity=0.8,
                   log_y=False,
                   color_discrete_sequence=['#2a9d8f'],
                   text_auto=True,
                   template='plotly_white',
                   width=800, 
                   height=400)
fig.update_layout(
    yaxis_title_text='Frequency',
    bargap=0.2, # gap between bars of adjacent location coordinates
    #margin=dict(l=20, r=20, t=20, b=20),
    #paper_bgcolor="gray"
)
fig.update_traces(textfont_size=9, textangle=0, textposition="outside", cliponaxis=False)
fig.show(renderer='notebook')

Plot the bit rate for the top 6 destination addresses of the IP that sends the most.

In [ ]:
from plot import plot_bit_rate

plot_bit_rate(top_6_ip_dest, max_ip_data, max_ip)

#### 4. Top 5 Destination IP (received bytes) and Top 5 Source IP (sent bytes)


In [ ]:
top_5_ip_dest = data_frame.groupby(["IP_DST"])[['length']].agg('sum').sort_values(by=['length'], ascending=False).head(5)

In [ ]:
from plot import plot_bar_addresses

plot_bar_addresses(top_5_ip_dest, title='Top 5 IP Destination Addresses', ylabel='Destinations', xlabel='Received Bytes')

In [ ]:
top_5_ip_src = data_frame.groupby(["IP_SRC"])[['length']].agg('sum').sort_values(by=['length'], ascending=False).head(5)
plot_bar_addresses(top_5_ip_src, title='Top 5 Source Addresses', xlabel='Bytes Sent', ylabel='Sources')

#### 5. Evaluate bitRate considering all the trace with 3 different sampling rate

In [ ]:
from plot import plot_total_bit_rate

plot_total_bit_rate(data_frame)

#### 6. GeoLocal Referenciation of the 5 sessions with the highest amount of traffic generated
A session is defined as a tuple of a source IP and destination IP address. We count the number of these tuples in the data frame to determine the top 5 sessions.

In [ ]:
from ip2geotools.databases.noncommercial import DbIpCity
import folium
from util import geo_infos

Get the source and destination IPs of the top 5 sessions.

In [ ]:
ip_grouped = copy.deepcopy(data_frame)
df_srcdst = list(zip(ip_grouped.IP_SRC, ip_grouped.IP_DST))
top_sessions = Counter(df_srcdst).most_common(5)
src_ips, dst_ips = zip(*map(lambda a: (a[0][0], a[0][1]), top_sessions))

Get the longitude and latitude of the IP pairs.

In [ ]:
src_geo, dst_geo = geo_infos(src_ips, dst_ips)

src_geo = pd.DataFrame(src_geo, columns=['latitude', 'longitude', 'region'])
dst_geo = pd.DataFrame(dst_geo, columns=['latitude', 'longitude', 'region'])

Create a world map and connect the source and destination locations for each session.

In [ ]:
flow_map = folium.Map([0, 0], zoom_start=2, tiles='Stamen Terrain')

for i in range(len(src_geo)):
    folium.Marker([src_geo.loc[i][0], src_geo.loc[i][1]], popup='<i>Mt. Hood Meadows</i>',
                icon=folium.Icon(color='green')).add_to(flow_map)
    folium.Marker([dst_geo.loc[i][0], dst_geo.loc[i][1]], popup='<i>Mt. Hood Meadows</i>',
                icon=folium.Icon(color='red')).add_to(flow_map)
    folium.PolyLine([(src_geo.loc[i][0], src_geo.loc[i][1]), (dst_geo.loc[i][0], dst_geo.loc[i][1])],
                  color="blue", weight=1.5, opacity=1).add_to(flow_map)

flow_map.save("./Map_top_5_flows.html")
display(flow_map)

#### 7. Flow Analysis - 10 Protocol mostly used

To find the most common protocols, we need to first find the flows in the data. A flow is a quintuplet of `source ip, destination ip, source port, destination port, protocol`. We then count the number of items in each group and sum them by protocol to get the total count.

In [ ]:
grouped_flows = data_frame.groupby(['IP_SRC', 'IP_DST', 'Protocol', 'src_port', 'dst_port']).agg(tot_len = pd.NamedAgg(column = 'length', aggfunc = 'sum')).reset_index()
number_to_protocol = {1:"ICMP", 6:"TCP", 17:"UDP", 50:"ESP", 4:"IPv4", 47:"GRE", 89:"OSPFIGP", 97:"ETHERIP", 103:"PIM"}
grouped_flows["Protocol"] = grouped_flows["Protocol"].replace(number_to_protocol)

Plot the most commonly used protocols in the data.

In [ ]:
from plot import plot_protocol_count

plot_protocol_count(grouped_flows)

### 8. Port Scanner evaluation (10 Ports mostly used)

In [ ]:
def port_count(ports):
    """Count the occurrences of well-known ports and return them as a dictionary."""
    well_known_ports = ports[ports < 1024]
    well_known_ports_count = well_known_ports.value_counts().reset_index()
    well_known_ports_count.columns = ['port', 'count']
    return well_known_ports_count

Find the number of occurrences of each port and sort them in descending order for the bar plot.

In [ ]:
source_ports = port_count(data_frame["src_port"])
dest_ports = port_count(data_frame["dst_port"])

source_ports.sort_values(by='count', ascending=False, inplace=True)
dest_ports.sort_values(by='count', ascending=False, inplace=True)

Add the protocol name to the number.

In [ ]:
port_number_to_name = {443:"443 - HTTPS",80:"80 - HTTP",-1:"-1 - ICMP",53:"53 - DNS",873:"873 - rsync",993:"993 - IMAP4",22:"22 -SSH",161:"161 - SNMP ",25:"25 - SMTP",123:"123 - NTP"}

dports_10 = dest_ports.sort_values(by=['count'], ascending=False).head(10)
dports_10["port"] = dports_10["port"].replace(port_number_to_name)

sports_10 = source_ports.sort_values(by=['count'], ascending=False).head(10)
sports_10["port"] = sports_10["port"].replace(port_number_to_name)

Plot the destination ports.

In [ ]:
port_plot = px.bar(dports_10,x=dports_10['port'],y=dports_10['count'],text_auto=True,
                    title='Top 10 Destination ports used',
                    opacity=0.8,
                    color_discrete_sequence=['#86bbd8'],
                    template='plotly_white',
                    width=800, 
                    height=400)

Plot the source ports.

In [ ]:
sport_plot = px.bar(sports_10,x=sports_10['port'],y=sports_10['count'],text_auto=True,
                    title='Top 10 Source ports used',
                    opacity=0.8,
                    color_discrete_sequence=['#86bbd8'],
                    template='plotly_white',
                    width=800, 
                    height=400)

In [ ]:
fig = make_subplots(rows=1, cols=2,subplot_titles=("Destination Ports", "Source Ports"),shared_yaxes=True)
fig.add_trace(port_plot['data'][0], row=1, col=1)
fig.add_trace(sport_plot['data'][0], row=1, col=2)
fig.update_layout(
    height=600, 
    width=1500,
    title_text="10 most used ports",
    xaxis_title_text ='Ports',
    yaxis_title_text ='Count')
fig.update_layout(template='plotly_white')
fig.update_xaxes(tickfont_family="Arial Black")
fig.update_xaxes(title_text="ports", row=1, col=2)
fig.update_yaxes(title_text="count", row=1, col=2)
fig.update_traces(textfont_size=9, textangle=0, textposition="outside", cliponaxis=False)

### 9. InterArrival Time boxplot between TCP and UDP Sessions

In [ ]:
import copy
from collections import Counter

def inter_arrival_time(data):
    val = np.array(data["time"])
    #Calculate the n-th discrete difference along the given axis
    return np.diff(val)

data_protocol = copy.deepcopy(data_frame[data_frame["Protocol"].isin([6,17])])
data_protocol["Protocol"] = data_protocol["Protocol"].replace({1:"ICMP",6:"TCP",17:"UDP"})

print(Counter(data_protocol["Protocol"]))

In [ ]:
box_len = px.box(data_protocol, y="length", x='Protocol', color='Protocol', template='plotly_white',color_discrete_sequence=[ '#e5b769' ,'#2a9d8f'])
#Seems like we have small pkts
#From the histogram at the beginning of the document we can see that almost all the pkts are < 2000 byte
#Let's try to do for only pkts of size smaller than the quantile 90%

In [ ]:
data_protocol_2000 = data_protocol[data_protocol["length"]<= 2860]
box_len_2000 = px.box(data_protocol_2000, y="length", x='Protocol', color='Protocol',template='plotly_white',color_discrete_sequence=[ '#e5b769' ,'#2a9d8f'])
# The TCP box is quite big so the interquantile range is large... so we have data that is quite variable
# The UDP's box is smaller that the TCP one so here the variance is smaller and also the size of the pckts is smaller that the TCP. UDP also does not have pckts pf size bigger that 1500 byte.

In [ ]:
np.quantile(np.array(data_protocol['length']),q=0.9) #our quantile

In [ ]:
fig = make_subplots(rows=1, cols=4,subplot_titles=("Original data UDP","Original data TCP", "Pckts whit length < 2860 bytes UDP","Pckts whit length < 2860 bytes TCP"))
fig.add_trace(box_len['data'][0], row=1, col=1)
fig.add_trace(box_len['data'][1], row=1, col=2)
fig.add_trace(box_len_2000['data'][0], row=1, col=3)
fig.add_trace(box_len_2000['data'][1], row=1, col=4)
fig.update_layout(
    height=600, 
    width=1500,
    title_text="Packets length")
fig.update_layout(template='plotly_white',showlegend=False)

We see that there are no UDP packets bigger than 1500 bytes, which is the Maximum Transmission Unit (MTU) on the link layer.

In [ ]:
soloUDP = data_protocol[data_protocol["Protocol"]=='UDP']
np.max(np.array(soloUDP['length']))

Inter arrival time.

In [ ]:
tcp_data = data_protocol_2000[data_protocol_2000["Protocol"]=="TCP"]
udp_data = data_protocol_2000[data_protocol_2000["Protocol"]=="UDP"]

inteArr_TCP= []
for elem in tcp_data.groupby(['IP_SRC', 'IP_DST', 'Protocol', 'src_port', 'dst_port']):
    #groupby tuple (key,dataframe)
    inteArr_TCP += inter_arrival_time(elem[1]).tolist()

inteArr_UDP = []
for elem in udp_data.groupby(['IP_SRC', 'IP_DST', 'Protocol', 'src_port', 'dst_port']):
    inteArr_UDP += inter_arrival_time(elem[1]).tolist()


val_ = inteArr_TCP + inteArr_UDP

label_TCP = [ "TCP" for i in range(len(inteArr_TCP))]
label_UDP =[ "UDP" for i in range(len(inteArr_UDP))]

lab_ = label_TCP + label_UDP

d = {'Protocol': lab_, 'IntArrTime': val_}
df = pd.DataFrame(data=d)

In [ ]:
print("Mean InterArrivalTime TCP Session: %.4f"% np.mean(np.array(inteArr_TCP)))
print("Mean InterArrivalTime UDP Session: %.4f"% np.mean(np.array(inteArr_UDP)))
#the TCP interarrival time is smaller than the UDP
#median?
print("Median InterArrivalTime TCP Session: %.5f"% np.median(np.array(inteArr_TCP)))
print("Median InterArrivalTime UDP Session: %.5f"% np.median(np.array(inteArr_UDP)))
# 3rd quartile
print("3rd quartile InterArrivalTime TCP Session: %.5f"% np.quantile(np.array(inteArr_TCP),q=0.75))
print("3rd quartile InterArrivalTime UDP Session: %.5f"% np.quantile(np.array(inteArr_UDP),q=0.75))
#After this make sense to plot it for values less that 0.00004 (median of TCP) seconds and 0.0009 (median of UDP)

In [ ]:
tcp_median = round(np.median(np.array(inteArr_TCP)),7)
udp_median = round(np.median(np.array(inteArr_UDP)),7)

In [ ]:
df1 = df[df["IntArrTime"] <  udp_median]
fig1 = px.box(df1, y="IntArrTime", x='Protocol', color='Protocol', template='plotly_white',color_discrete_sequence=[ '#e5b769' ,'#2a9d8f'])
#fig1.show()

In [ ]:
df2 = df[df["IntArrTime"] < tcp_median]
fig2 = px.box(df2, y="IntArrTime", x='Protocol', color='Protocol', template='plotly_white',color_discrete_sequence=[ '#e5b769' ,'#2a9d8f'])
#fig2.show()

In [ ]:
fig = make_subplots(rows=1, cols=4,subplot_titles=(f"TCP < {udp_median} s", f"UDP < {udp_median} s",f"TCP < {tcp_median}", f"UDP < {tcp_median}"))
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig1['data'][1], row=1, col=2)
fig.add_trace(fig2['data'][0], row=1, col=3)
fig.add_trace(fig2['data'][1], row=1, col=4)
fig.update_layout(
    height=600, 
    width=1500,
    title_text="Inter Arrival Time",
    yaxis_title_text='Time (s)')
fig.update_layout(template='plotly_white',showlegend=False)

### 10. Develop your own analysis (e.g. Topology of the network using networkx)

In [ ]:
import networkx as nx
import random
from networkx.algorithms import approximation as apx
random.seed(26111998)

In [ ]:
Dest_IPs, Source_IPs = list(data_frame['IP_DST']),list(data_frame['IP_SRC'])
len(np.unique(Dest_IPs+Source_IPs))

In [ ]:
data_udp=data_frame[data_frame['Protocol']==17]
data_tcp=data_frame[data_frame['Protocol']==6]
data_ICMP=data_frame[data_frame['Protocol']==1]
data_transport=data_frame[(data_frame['Protocol']==17) | (data_frame['Protocol']==6)]

In [ ]:
Dest_IPs_UDP,Source_IPs_UDP=list(data_udp['IP_DST']),list(data_udp['IP_SRC'])
print("There are ",len(np.unique(Dest_IPs_UDP+Source_IPs_UDP))," nodes sending and receiving UDP pckts")
Dest_IPs_TCP,Source_IPs_TCP=list(data_tcp['IP_DST']),list(data_tcp['IP_SRC'])
print("There are ",len(np.unique(Dest_IPs_TCP+Source_IPs_TCP))," nodes sending and receiving TCP pckts")
Dest_IPs_ICMP,Source_IPs_ICMP=list(data_ICMP['IP_DST']),list(data_ICMP['IP_SRC'])
len(np.unique(Dest_IPs_ICMP+Source_IPs_ICMP))
print("There are ",len(np.unique(Dest_IPs_ICMP+Source_IPs_ICMP)),"nodes sending and receiving ICMP pckts")

#### UDP GRAPH

In [ ]:
data_udp = data_udp.groupby(['IP_SRC', 'IP_DST', 'src_port', 'dst_port']).first().reset_index()
data_udp = data_udp[['IP_SRC', 'IP_DST', 'src_port', 'dst_port']]
UDP=data_udp[['IP_SRC','IP_DST']]
udp_count=dict(UDP.value_counts())
l=[]
for i in range(len(data_udp)):
    l.append(udp_count[(data_udp.iloc[i]['IP_SRC'],data_udp.iloc[i]['IP_DST'])])
data_udp['Num Flows']=l
data_udp


In [ ]:
Graph_UDP=nx.DiGraph()

for _,i in data_udp.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    if (node_a,node_b) in Graph_UDP.edges:
        Graph_UDP.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'])]
    else:
        Graph_UDP.add_edge(node_a,node_b)
        Graph_UDP.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_UDP.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'])]

#### TCP GRAPH

In [ ]:
data_tcp = data_tcp.groupby(['IP_SRC', 'IP_DST', 'src_port', 'dst_port']).first().reset_index()
data_tcp = data_tcp[['IP_SRC', 'IP_DST', 'src_port', 'dst_port']]
TCP=data_tcp[['IP_SRC','IP_DST']]
tcp_count=dict(TCP.value_counts())
l=[]
for i in range(len(data_tcp)):
    l.append(tcp_count[(data_tcp.iloc[i]['IP_SRC'],data_tcp.iloc[i]['IP_DST'])])
data_tcp['Num Flows']=l

In [ ]:
Graph_TCP = nx.DiGraph()

for _,i in data_tcp.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    if (node_a,node_b) in Graph_TCP.edges:
        Graph_TCP.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'])]
    else:
        Graph_TCP.add_edge(node_a,node_b)
        Graph_TCP.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_TCP.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'])]

#### ICMP GRAPH

In [ ]:
data_ICMP = data_ICMP.groupby(['IP_SRC', 'IP_DST', 'src_port', 'dst_port']).first().reset_index()
data_ICMP = data_ICMP[['IP_SRC', 'IP_DST', 'src_port', 'dst_port']]
ICMP=data_ICMP[['IP_SRC','IP_DST']]
ICMP_count=dict(ICMP.value_counts())
l=[]
for i in range(len(data_ICMP)):
    l.append(ICMP_count[(data_ICMP.iloc[i]['IP_SRC'],data_ICMP.iloc[i]['IP_DST'])])
data_ICMP['Num Flows']=l
data_ICMP

In [ ]:
Graph_ICMP = nx.DiGraph()

for _,i in data_ICMP.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    if (node_a,node_b) in Graph_ICMP.edges:
        Graph_ICMP.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'])]
    else:
        Graph_ICMP.add_edge(node_a,node_b)
        #In this case we could simply go for a directed graph without attributes because the numflow is always 1 and the list is [-1,-1] for all
        Graph_ICMP.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_ICMP.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'])]

### Visualization of the topology of the 3 graphs

In [ ]:
sample_UDP=data_udp.sample(400)
sample_TCP=data_tcp.sample(400)
sample_ICMP=data_ICMP.sample(400)

In [ ]:
Graph_subset_UDP=nx.DiGraph()

for _,i in sample_UDP.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    if (node_a,node_b) in Graph_subset_UDP.edges:
        Graph_subset_UDP.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'])]
    else:
        Graph_subset_UDP.add_edge(node_a,node_b)
        Graph_subset_UDP.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_subset_UDP.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'])]

Graph_subset_TCP=nx.DiGraph()

for _,i in sample_TCP.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    if (node_a,node_b) in Graph_subset_TCP.edges:
        Graph_subset_TCP.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'])]
    else:
        Graph_subset_TCP.add_edge(node_a,node_b)
        Graph_subset_TCP.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_subset_TCP.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'])]

Graph_subset_ICMP=nx.DiGraph()

for _,i in sample_ICMP.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    if (node_a,node_b) in Graph_subset_ICMP.edges:
        Graph_subset_ICMP.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'])]
    else:
        Graph_subset_ICMP.add_edge(node_a,node_b)
        Graph_subset_ICMP.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_subset_ICMP.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'])]


In [ ]:
plt.style.use('_mpl-gallery')
fig, axes = plt.subplots(1,3, figsize=(15,4))
nx.draw(Graph_subset_UDP,node_size = 20, width = 0.5, node_color = '#2a9d8f', ax=axes[0])
nx.draw(Graph_subset_TCP,node_size = 20, width = 0.5, node_color = '#2a9d8f', ax=axes[1])
nx.draw(Graph_subset_ICMP,node_size = 20, width = 0.5, node_color = '#2a9d8f', ax=axes[2])

axes[0].set_title("UDP sample Graph")
axes[1].set_title("TCP sample Graph")
axes[2].set_title("ICMP sample Graph")

plt.show()


After this overview of the topology we have decided to put together the TCP and UDP packets since the graphs do seem to be very similar, while the ICMP graph has a particular topology with fewer sources and more destinations than the other two protocols. <br>
This is beacause the ICMP protocol is used for troubleshooting, the sources want to know if the destinations are reacheable.

### Transport GRAPH

Here, as we have just mentioned above we are going to build a single graph containing both the packets send using the TCP or UDP protocols.

In [ ]:
# So let's merge them!
data_transport = data_transport.groupby(['IP_SRC', 'IP_DST', 'src_port', 'dst_port',"Protocol"]).first().reset_index()
data_transport = data_transport[['IP_SRC', 'IP_DST', 'src_port', 'dst_port',"Protocol"]]
TR=data_transport[['IP_SRC','IP_DST']]
tr_count=dict(TR.value_counts())
l=[]
for i in range(len(data_transport)):
    l.append(tr_count[(data_transport.iloc[i]['IP_SRC'],data_transport.iloc[i]['IP_DST'])])
data_transport['Num Flows']=l
#data_transport

In [ ]:
# Creating the new graph for both TCP and UDP
Graph_Transport=nx.DiGraph()
prot={17:'UDP',6:'TCP'}
for _,i in data_transport.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    p=prot[i.Protocol]
    if (node_a,node_b) in Graph_Transport.edges:
        Graph_Transport.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'],p)]
    else:
        Graph_Transport.add_edge(node_a,node_b)
        Graph_Transport.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_Transport.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'],p)]

In [ ]:
#Graph_Transport.get_edge_data("95.36.218.85","202.9.24.18")

And now let's observe a small sample of the graph obtained combining both UDP and TCP pkts.

In [ ]:
transport_sample = data_transport.sample(400)

In [ ]:
np.unique(transport_sample.Protocol)

In [ ]:
Graph_Subset_Transport=nx.DiGraph()
prot={17:'UDP',6:'TCP'}
for _,i in transport_sample.iterrows():
    node_a=i['IP_SRC']
    node_b=i['IP_DST']
    p=prot[i.Protocol]
    if (node_a,node_b) in Graph_Subset_Transport.edges:
        Graph_Subset_Transport.edges[node_a,node_b]['List']+=[(i['src_port'],i['dst_port'],p)]
    else:
        Graph_Subset_Transport.add_edge(node_a,node_b)
        Graph_Subset_Transport.edges[node_a,node_b]['Num Flow']=i['Num Flows']
        Graph_Subset_Transport.edges[node_a,node_b]['List']=[(i['src_port'],i['dst_port'],p)]

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))
nx.draw(Graph_Subset_Transport,node_size = 20, width = 0.5, node_color = '#2a9d8f', font_size = 6,ax=ax)
plt.title("Sample of the Transport Graph")
plt.show()

As we have observed before, the overall topology is very similar to the two graphs that we saw before. Hence, we think that, in terms of the topological study, it makes more sense to study the entirety of the pkt that use TCP and UDP altogether. 

In [ ]:
e1,e2=transport_sample[transport_sample['Num Flows']>1].iloc[0][['IP_SRC','IP_DST']]

In [ ]:
Graph_Subset_Transport.get_edge_data(e1, e2)
# Here we can observe how now every flow has both assigned the source and the destination port plus the protocol used

### Different Metrics to obtain a better understanding of the Graphs' topologies

In [ ]:
# How many nodes we have?
print("Number of nodes: %.0f"% nx.number_of_nodes(Graph_ICMP))
print("Number of edges: %.0f"% nx.number_of_edges(Graph_ICMP))


In [ ]:
# Degree of the two Graphs
Counter(nx.degree_histogram(Graph_ICMP)) #almost all the nodes have degree 0
#The degree centrality for a node v is the fraction of nodes it is connected to.
plt.style.use('_mpl-gallery')
plt.rcParams['font.family'] = 'Serif'
plt.figure(figsize=(4, 4))
#fig, axes = plt.subplots(1,2, figsize=(15,4))
icmp_hist = plt.hist(np.log(list(nx.degree_centrality(Graph_ICMP).values())),bins=30,label='ICMP')
transport_hist = plt.hist(np.log(list(nx.degree_centrality(Graph_Transport).values())),bins=30,label='Transport')
plt.xlabel("Degree Centrality")
plt.legend()
plt.title("Degree in log-scale")
plt.show()
# there are lot of nodes that are destination and have a degree centrality near 0.
# Nonetheless there are very few nodes with higher degree centrality

In [ ]:
print("The mean of the degree of the Transport Graph is %.6f"% np.mean(list(nx.degree_centrality(Graph_Transport).values())),"and the variance is %.6f"% np.var(list(nx.degree_centrality(Graph_Transport).values())))
print("The mean of the degree of the ICMP Graph is %.6f"% np.mean(list(nx.degree_centrality(Graph_ICMP).values())),"and the variance is %.6f"% np.var(list(nx.degree_centrality(Graph_ICMP).values())))
print()
ratio = np.var(list(nx.degree_centrality(Graph_ICMP).values()))/np.var(list(nx.degree_centrality(Graph_Transport).values()))
print("The ratio between the two variances is %.2f"% ratio ) 

In [ ]:
print("How many nodes we need to delete to have a disconnected graph?", apx.node_connectivity(Graph_ICMP)) # The graph is not connected and we can not calculate algorithms like the longest path
# Node connectivity is equal to the minimum number of nodes that must be removed to disconnect G or render it trivial. 
# By Menger’s theorem, this is equal to the number of node independent paths (paths that share no nodes other than source and target).

Our graphs are disconnected and directed, in order to evaluate other metrics we construct the undirected versions of the two graphs

In [ ]:
# UNDIRECTED GRAPHS
undirected_ICMP = nx.Graph()
undirected_ICMP.add_edges_from(Graph_ICMP.edges())
undirected_transport = nx.Graph()
undirected_transport.add_edges_from(Graph_Transport.edges()) 

In [ ]:
print("Is the ICMP graph acyciclic?", nx.is_directed_acyclic_graph(Graph_ICMP)) # so our graph have cycles and we can not do the longest path (also the other 2)
print("Is the Transport graph acyciclic?", nx.is_directed_acyclic_graph(Graph_Transport)) 
print("Are there some clusters in ICMP?", apx.average_clustering(undirected_ICMP)) #no clutsers in ICMP
print("Are there some clusters in Transport graph?", apx.average_clustering(undirected_transport)) #neither for the transport data

In [ ]:
# Let's calculate the number of different components
print("Number of Connected Components of ICMP Graph is", nx.number_connected_components(undirected_ICMP), "and the number of connected components of the Transport Graph is",nx.number_connected_components(undirected_transport))
#ok, now we can look at the set of nodes in the connected graph containing the source that send more (from point 3)
print("The set of nodes in the component of the ICMP Graph cointaing node 150.57.136.251 are", nx.node_connected_component(undirected_ICMP,'150.57.136.251')) #only 4 nodes
print("The cardinality of set of nodes in the component of the Transport Graph cointaing node 150.57.136.251 are", len(nx.node_connected_component(undirected_transport,'150.57.136.251'))) #lot of nodes, let's display how many they are

In [ ]:
# Now we can look at the diameter of the components
# In graph theory, the diameter of a connected component refers to the longest shortest path between any two nodes within that component. 
# In other words, it measures the maximum number of edges that must be traversed to go from one node to another within the component.
component_diameter_ICMP = []
for component in nx.connected_components(undirected_ICMP):
    component_diameter_ICMP.append(nx.diameter(undirected_ICMP.subgraph(component)))

In [ ]:
component_diameter_T = []
for component in nx.connected_components(undirected_transport):
    component_diameter_T.append(nx.diameter(undirected_transport.subgraph(component)))

In [ ]:
#Prepare the Bar Plot for both the ICMP and the Transport Diameters
#ICMP
ICMPx=list(range(min(component_diameter_ICMP),max(component_diameter_ICMP)+1))
aux=dict(sorted(Counter(component_diameter_ICMP).items()))
for i in ICMPx:
    if i not in aux.keys():
        aux[i]=0
ICMPy=list(dict(sorted(aux.items())).values())

#Transport
Tx=list(range(min(component_diameter_T),max(component_diameter_T)+1))
auxT=dict(sorted(Counter(component_diameter_T).items()))
for i in Tx:
    if i not in auxT.keys():
        auxT[i]=0
auxT[0]=0
Tx.append(0)
Tx=sorted(Tx)
Ty=list(dict(sorted(auxT.items())).values())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5),sharey=False)

# Bar Plot ICMP diameter nel primo subplot
ax1.bar(ICMPx, ICMPy, label="ICMP diameter")
ax1.set_xlabel("Diameter")
ax1.set_ylabel("Frequency")
ax1.set_title("Frequencies of ICMP Component's Diameters")

# Bar Plot Transport diameter nel secondo subplot
ax2.bar(Tx,Ty, label="Transport diameter", color='orange')
ax2.set_xlabel("Diameter")
ax2.set_ylabel("Frequency")
ax2.set_title("Frequencies of Transport Component's Diameters - Log Scale")
ax2.set_xticks(range(len(Tx)), Tx)
ax2.set_yscale("log")
#plt.savefig("out/")
plt.tight_layout()
plt.show()

# Part B

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.cluster import KMeans

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

from kneed import KneeLocator
import math
import operator 


In [ ]:
data_frame["Label DSCP"] = pd.to_numeric(data_frame["Label DSCP"])

In [ ]:
dscp_tab = {0: "BE",
            8: "Priority",
            10: "Priority",
            12: "Priority",
            14: "Priority",
            16: "Immediate",
            18: "Immediate",
            20: "Immediate",
            22: "Immediate",
            24: "Flash voice",
            26: "Flash voice",
            28: "Flash voice",
            30: "Flash voice",
            32: "Flash Override",
            34: "Flash Override",
            36: "Flash Override",
            38: "Flash Override",
            40: "Critical voice RTP",
            46: "Critical voice RTP",
            48: "Internetwork control",
            56: "Network Control"
            } 

data_frame = data_frame.replace({'Label DSCP': dscp_tab})
data_frame = data_frame.replace({'Label DSCP': {"Priority":"AF","Immediate":"AF","Flash voice":"AF",
                                "Flash Override":"AF","Critical voice RTP":"EF",
                                 "Internetwork control":"CS6","Network Control":"CS6",
                                 4:"NotKnown",2:"NotKnown",6:"NotKnown",7:"NotKnown",
                                 1:"NotKnown",41:"EF",42:"EF",43:"EF",44:"EF",45:"EF",49:"NotKnown",54:"NotKnown",11:"NotKnown",50:"NotKnown",29:"NotKnown"}})

print("DSCP Occurrences: ")
print(dict(Counter(data_frame["Label DSCP"])))

In [ ]:
data_frame

In [ ]:
data_unique = data_frame.drop_duplicates(["IP_DST","dst_port"])

#all possible (IP_0,port_0)
flows_list = data_unique[["IP_DST","dst_port"]].values.tolist()

dict_rows = {}
for i in tqdm(range(len(flows_list))):
    #extract all packets received by each specific couple IP dst, port destination
    subdata = data_frame[(data_frame["IP_DST"] == flows_list[i][0]) & (data_frame["dst_port"] == flows_list[i][1])]
    
    #20 is just the length of our vector when we change the values in a logaritmic scale
    #max 2**19 --> 524288 | This consideration depends on your dataset
    length = np.zeros(21)
    pkt = np.zeros(21)
    
    #At least 2 pkts received by this specific (IP_0,port_0)
    if subdata.shape[0] >= 2:
        
        

        #Check about the label, we want to be sure to analyze a couple with just 1 DSCP
        #The vector that represents this element will have just one label
        
        if Counter(subdata["Label DSCP"] == 1):
        
            dtu = subdata.drop_duplicates(["IP_SRC","src_port"])
            
            list_couple_src = dtu[["IP_SRC","src_port"]].values.tolist()
            # New features:
            # Packet level: min, max, mean and variance of pkt lenght.
            # TO DO:
            # Flow level: statistics of number of packets per flow, number of bytes per flow, flow duration, interarrival times
            # TCP connection level: statistics of packets per TCP connection, bytes per connection, connection duration

            
            minimum,maximum=np.infty,0

            for elem in list_couple_src:
                #Observe each element in the Neighborhood (N)
                finaldata = subdata[(subdata["IP_SRC"]==elem[0]) & (subdata["src_port"]==elem[1])]

                aux1,aux2=min(finaldata['length']),max(finaldata['length'])
                if aux1<minimum:
                    minimum=aux1
                if aux2>maximum:
                    maximum=aux2

                mean = np.mean(finaldata['length'])
                var = np.var(finaldata['length'])

                #this is for avoiding -inf values because the log of 0 is not defined
                if (var == 0):
                    var = 1
                if (mean == 0):
                    mean = 1
                    

                #Number of packets
                #Ex: pck = 245, log_{2}(245) = 7.94 --> ceil()--> 8 
                #The range considered is (2**7,2**8] = (128,256]
                length[math.ceil(math.log(finaldata.shape[0])/math.log(2))] += 1
                
                #Packet length analysis --> Byte
                #extract each packet length
                for index,row in finaldata.iterrows():
                    pkt[math.ceil(math.log(row["length"])/math.log(2))] += 1
                    
            # Normalization vector both for packets and bytes    
            # Tring to transform the new features in the log2 scale
            dict_rows[(flows_list[i][0],flows_list[i][1])] = [list(Counter(subdata["Label DSCP"]).keys())[0],length/sum(length),pkt/sum(pkt),np.log2(maximum),np.log2(minimum),np.log2(mean),np.log2(var)]
            
        else:
            #print("problem")
            break

            
#Save the data in a pickle file
aux=pd.DataFrame.from_dict(dict_rows)
pd.to_pickle(aux,'flows.pkl')

In [ ]:
dataFlow=pd.read_pickle('flows.pkl')
dataFlow.T

In [ ]:
data_pandas = []        
for k,val in dataFlow.items():
    obs = []
    obs.append(val[0])
    obs.extend(val[1].tolist())
    obs.extend(val[2].tolist())
    obs.append(val[3])
    obs.append(val[4])
    obs.append(val[5])
    obs.append(val[6])
    data_pandas.append(obs)
col = ["Label"]
col.extend(["X"+str(i)for i in range(46)])
dataUns = pd.DataFrame.from_records(data_pandas,columns=col )
#Select just items with a string label and not numeric #mmmm we have all string 
#dataUns = dataUns[dataUns["Label"].isin(['AF','BE', 'CS6','EF','NotKnown'])]
#Useful to encode the label, it will be exploited at the end of the classification
le = preprocessing.LabelEncoder() #here we are transforming "BE" and the other labels into numbers
dataUns["Label"]  = le.fit_transform(dataUns["Label"])

In [ ]:
#Extract X,Y
X = dataUns.iloc[:,1:]
Y = dataUns.iloc[:,0]

# Step:
# 1)Extract train and test from our starting dataset


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size = 0.30, random_state = 0)

# 2) Apply oversampling to rebalance in the training the number of occurrences
Not oversampling the 1, is the BE, we have already loooots of them, make all balanced classes, same cardinality as class with label 1.

In [ ]:
oversample = SMOTE(sampling_strategy={0:sum(y_train==1),
                                      2:sum(y_train==1),
                                      4:sum(y_train==1),
                                      3:sum(y_train==1)},k_neighbors=2) 
X_over, Y_over = oversample.fit_resample(x_train, y_train)

before_oversampling = Counter(le.inverse_transform(y_train))

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for ax, sample, title in zip(axes, [Counter(le.inverse_transform(y_train)), Counter(le.inverse_transform(Y_over))], ["Before", "After"]):
    ax.bar(x=list(sample.keys()), height=sample.values())
    ax.set_yscale("log")
    ax.set_title(title)

fig.suptitle("Oversampling")
fig.savefig("out/oversampling.png", format="png")

In [ ]:
kmeans_kwargs = {
    "init": "k-means++",
    "n_init": 10,
    "max_iter": 1000,
    "random_state": 26111998
}

In [ ]:
#3)Find the optimal K (number of clusters) according to the training
sse = []
for k in range(1, 15):
    kmeans = KMeans(n_clusters=k,**kmeans_kwargs)
    kmeans.fit(X_over)
    sse.append(kmeans.inertia_)

kl = KneeLocator(
    range(1, 15), sse, curve="convex", direction="decreasing", interp_method= "interp1d"
)
opt = kl.elbow


In [ ]:
plt.plot(range(1, 15), sse)
plt.xticks(range(1, 15))
plt.vlines(x=opt, ymin=0, ymax=max(sse), linestyles="dashed")
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("Elbow Method")
plt.savefig("out/elbow_method.png", format="png")
plt.show()

In [ ]:
#Extract the minimum in the convex curve 
kl = KneeLocator(
    range(1, 15), sse, curve="convex", direction="decreasing", interp_method= "interp1d"
)
opt = kl.elbow 
print("Optimal number of clusters: ",opt)

In [ ]:
#4) Apply this clustering to the test
#Apply again K.Means with this specific number of clusters
kmeans = KMeans(
    init="k-means++",
    n_clusters=opt,
    n_init=10,
    max_iter=1000,
    random_state=26111998
)
kmeans.fit(X_over)

In [ ]:
#Observe the results
#In each cluster finding the occurrences of the DSCP Labels

dict_label_dscp = {}

for i in list(set(kmeans.labels_)):
    #print(sum(kmeans.labels_== i))
    ind = []
    for s, j in enumerate(kmeans.labels_):
        if j == i:
            ind.append(s) 

    print("Label: ",i)
    stats = Counter(le.inverse_transform(Y_over[ind]))
    print(stats)
    print(max(stats.items(), key=operator.itemgetter(1))[0])
    dict_label_dscp[i] = max(stats.items(), key=operator.itemgetter(1))[0]
    print()
    print()

In [ ]:
#Test
pred = kmeans.predict(x_test)    
prediction = [ dict_label_dscp[elem] for elem in pred ]    

In [ ]:
labels = [ "BE", "NotKnown","AF", "EF","CS6"]
def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.gray_r):
    
    '''Confusion Matrix Evaluation'''
    
    plt.figure(figsize=(5,5))
    plt.matshow(df_confusion, cmap=cmap,fignum=1) # imshow
    
    for (i, j), z in np.ndenumerate(df_confusion):
        plt.text(j, i, '{:0.2f}'.format(z), ha='center', va='center',
                 bbox=dict(boxstyle='round', facecolor='white'))
    
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45,fontsize = 13)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(tick_marks, df_confusion.index,fontsize = 13)
    #plt.tight_layout()
    #plt.ylabel(df_confusion.index.name)
    #plt.xlabel(df_confusion.columns.name)
    plt.ylabel("True",fontsize = 18)
    plt.xlabel("Predicted",fontsize = 18)
    plt.grid(False)
    plt.savefig("out/confusion_matrix.png", format="png", bbox_inches="tight")
    plt.show()

In [ ]:
# Confusion matrix evaluation   
confmatrix = confusion_matrix(le.inverse_transform(y_test), 
                              prediction,
                              labels=labels)

df_confusion = pd.DataFrame(confmatrix, index=labels, columns=labels)
df_conf_norm = df_confusion.div(df_confusion.sum(axis=1),axis=0)

plot_confusion_matrix(df_conf_norm,cmap=plt.cm.YlOrBr)

In [ ]:
sample_silhouette_values = silhouette_samples(X_over, kmeans.labels_)
print("Average silhouette score:", sample_silhouette_values)

In [ ]:
colors = ['#ff6b35', '#f7c59f', '#efefd0', '#004e89', '#1a659e']

In [ ]:
def silhouette_plot(clusters):

    fig, ax = plt.subplots()

    y_lower = 10

    for i in range(clusters.n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[clusters.labels_ == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = colors[i % len(colors)]
        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  # Update the value for the next graph


    # axes limits
    ax.set_ylim(0, len(X_over) + (opt + 1) * 10)
    ax.set_xlabel("Silhouette")
    ax.set_ylabel("Cluster")
    ax.set_title("Silhouette Scores")

    # Vertical blue line for the mean value
    silhouette_avg = silhouette_score(X_over, clusters.labels_)
    ax.axvline(x=silhouette_avg, color='#004e89', linestyle="--")

    # Text for the mean value
    ax.text(silhouette_avg + 0.01, 0, "Mean = {:.2f}".format(silhouette_avg))
    fig.savefig(f"out/silhouette_plot_{clusters.n_clusters}.png", format="png")
    # Show
    return fig, ax, silhouette_avg



In [ ]:
fig = silhouette_plot(kmeans)[0]
fig.show()

In [ ]:
fits = []

for num_clusters in range(2, 15):
    kmeans = KMeans(
        init="k-means++",
        n_clusters=num_clusters,
        n_init=10,
        max_iter=1000,
        random_state=26111998
    )
    kmeans.fit(X_over)
    fits.append(kmeans)


avg_silhouette_scores = [silhouette_score(X_over, kmeans.labels_) for kmeans in fits]

Plot the average silhouette scores.

In [ ]:
fig, ax = plt.subplots()

ax.plot(range(2, 15), avg_silhouette_scores)
ax.set_xlabel("Clusters")
ax.set_ylabel("Average Silhouette Score")
ax.set_title("Average Silhouette Scores")
fig.savefig("out/avg_silhouette.png", format="png")

#### Davies Bouldin Index
The score is defined as the average similarity measure of each cluster with its most similar cluster, where similarity is the ratio of within-cluster distances to between-cluster distances. Thus, clusters which are farther apart and less dispersed will result in a better score.

The minimum score is zero, with lower values indicating better clustering.

In [ ]:
from sklearn.metrics import davies_bouldin_score

def cluster_scores(score_func):
    db_scores = []
    n_clusters = range(2, 15)
    for k in n_clusters:
        kmeans = KMeans(n_clusters=k,**kmeans_kwargs)
        kmeans.fit(X_over)
        db_score = score_func(X_over, kmeans.labels_)
        db_scores.append(db_score)

    return n_clusters, db_scores

def cluster_score_plot(n_clusters, scores, title):

    fig, ax = plt.subplots()

    ax.plot(n_clusters, scores)
    ax.set_xlabel("Number of Clusters")
    ax.set_ylabel("Score")
    ax.set_title(title)
    return fig, ax

n_clusters, db_scores = cluster_scores(davies_bouldin_score)
fig, _ = cluster_score_plot(n_clusters, db_scores, "Davies Bouldin")
fig.savefig("out/davies_bouldin.png", format="png")
fig.show()

#### Calinski Harabasz score
The score is defined as ratio of the sum of between-cluster dispersion and of within-cluster dispersion. It is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). Here cohesion is estimated based on the distances from the data points in a cluster to its cluster centroid and separation is based on the distance of the cluster centroids from the global centroid.

Higher value of CH index means the clusters are dense and well separated, though there is no rule for choosing the number of clusters.

In [ ]:
from sklearn.metrics import calinski_harabasz_score

n_clusters, ch_scores = cluster_scores(calinski_harabasz_score)
fig, _ = cluster_score_plot(n_clusters, ch_scores, "Calinski Harabasz")
fig.savefig("out/calinski_harabasz.png", format="png")
fig.show()